Import libraries

In [154]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
# used links:
#https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722
#https://www.datacamp.com/community/tutorials/web-scraping-using-python

Use beautiful soup to request the table from Wikipedia page and create a dataframe

In [155]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = urlopen(url)
#print(soup.prettify())
soup = BeautifulSoup(html, 'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
rows = My_table.find_all('tr')
list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
table = pd.DataFrame(list_rows)

Data preparation step

In [171]:
table1 = table[0].str.split(',', expand=True)
table1[2] = table1[2].str.strip('\n]')
table1[0] = table1[0].str.strip('[')
table1 = table1.iloc[1:]
table1.columns = ['Postcode','Borough', 'Neighbourhood']
table1['Postcode'] = table1['Postcode'].str.strip()
table1['Borough'] = table1['Borough'].str.strip()
table1['Neighbourhood'] = table1['Neighbourhood'].str.strip()
table2 = table1[table1['Borough'] != 'Not assigned']
table3 = table2.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))
table3['Neighbourhood'] = np.where(table3['Neighbourhood'] == 'Not assigned', table3['Borough'], table3['Neighbourhood'])

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


Call df shape

In [188]:
table3.shape 

(103, 3)